In [ ]:
import sys

# !{sys.executable} -m pip install wordcloud Pillow nltk lxml

In [4]:
import pandas as pd
import numpy as np
import re
import os
from collections import Counter
import glob
import zipfile
import re
import nltk
from nltk.tokenize import RegexpTokenizer

HOME_DIR = "/home_remote"
# HOME = "/home/thi.tra.my.nguyen"

In [ ]:
def datacollection(wpattern="./*.zip"):
    df = pd.DataFrame()
    for g in glob.glob(wpattern):
        zf = zipfile.ZipFile(g)
        dfs = [pd.read_xml(zf.open(f)) for f in zf.namelist()[1:]]
        df = pd.concat(dfs, ignore_index = True)

    return df

test = datacollection(os.path.join(HOME_DIR, "eRisk2018training/2017_test/*.zip"))

In [ ]:
def file_concat(path):
    train_neg = pd.DataFrame()
    for i in os.listdir(path):
        #print(i)
        allFiles = glob.glob(path+"/" + i+ "/*.xml")
        dt = []
        for x in allFiles:
            try:
                df = pd.read_xml(x)
                dt.append(df)
                #print(dt)
                train_neg = pd.concat(dt)
            except:
                print("This file error " + x)
    return train_neg

In [ ]:
train_neg = file_concat(os.path.join(HOME_DIR, "eRisk2018training/2017_train/negative_examples_anonymous_chunks"))
train_pos = file_concat(os.path.join(HOME_DIR, "eRisk2018training/2017_train/positive_examples_anonymous_chunks"))

In [ ]:
train_neg

### EDA

#### Word Cloud

In [ ]:
train_pos = train_pos.dropna(subset=['TEXT'])
train_neg = train_neg.dropna(subset=['TEXT'])

In [ ]:
#reference: https://docs.python.org/3/library/re.html and 
#https://thinkinfi.com/fasttext-word-embeddings-python-implementation/
def process_text(document):

        # Remove extra white space from text
        document = re.sub(r'\s+', ' ', document, flags=re.I)
         
        # Remove all the special characters from text
        document = re.sub(r'\W', ' ', str(document))
 
        # Remove all single characters from text
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
 
        # Converting to lowercase
        document = document.lower()
 
        return document

#reference: https://www.nltk.org/api/nltk.tokenize.regexp.html
cleaned_train = [process_text(sentence) for sentence in train_neg['TEXT'] if sentence.strip() !='']
tokenizer_train = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
train_token_neg= [tokenizer_train.tokenize(sent) for sent in cleaned_train]

cleaned_test = [process_text(sentence) for sentence in train_pos['TEXT'] if sentence.strip() !='']
train_token_pos = [tokenizer_train.tokenize(sent) for sent in cleaned_test]


In [ ]:
#flatten the list train_token_neg and train_token_pos
train_token_neg = [item for sublist in train_token_neg for item in sublist]
train_token_pos = [item for sublist in train_token_pos for item in sublist]

In [ ]:
#remove stopwords in train_token_neg and train_token_pos
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
train_token_neg = [w for w in train_token_neg if not w in stop_words]
train_token_pos = [w for w in train_token_pos if not w in stop_words]

#remove duplicates
train_token_neg = list(dict.fromkeys(train_token_neg))
train_token_pos = list(dict.fromkeys(train_token_pos))

#remove words with length less than 3
train_token_neg = [w for w in train_token_neg if len(w) >= 3]
train_token_pos = [w for w in train_token_pos if len(w) >=3]

#remove words are popular with frequency more than 10 times in both positive and negative
freq_neg = pd.Series(' '.join(train_neg['TEXT']).split()).value_counts()[:50]
freq_pos = pd.Series(' '.join(train_pos['TEXT']).split()).value_counts()[:50]
freq_neg = list(freq_neg.index)
freq_pos = list(freq_pos.index)
train_token_neg = [w for w in train_token_neg if w not in freq_pos]
train_token_pos = [w for w in train_token_pos if w not in freq_neg]


train_token_neg = [w for w in train_token_neg if w not in ['college', 'hear','seem','cup']]
train_token_pos = [w for w in train_token_pos if w not in ['call','economic','american','one','law']]

In [ ]:
#wordcloud for train_token_final and test_token_final
import matplotlib.pyplot as plt
from wordcloud import WordCloud

text1 = " ".join(train_token_neg)
text2 = " ".join(train_token_pos)

# Create and generate a word cloud image:
font_path = os.path.join(HOME_DIR, 'fonts/Arial.ttf')
wordcloud_neg= WordCloud(width=1000, height=600, background_color='white', font_path=font_path).generate(text1)
wordcloud_pos= WordCloud(width=1000, height=600, background_color='white', font_path=font_path).generate(text2)
plt.figure(figsize=(10,6))
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.axis("off")
plt.show()
plt.figure(figsize=(10,6))
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.axis("off")
plt.show()
#save picture as pdf
wordcloud_neg.to_file(os.path.join(HOME,  'wordcloud_neg.pdf'))
wordcloud_pos.to_file(os.path.join(HOME, 'wordcloud_pos.pdf'))


### Uni, Bi Gram and Inforamtion Gain

In [5]:
%store -r df
%store -r df_neg
df['Label'] = 1
df_neg['Label'] = 0

In [7]:
df

,User,Date,Title,Text,Label
0,train_subject4879,2014-02-15 05:03:58,,You forgot it in people- broken social scene\n...,1
1,train_subject4879,2014-02-11 04:55:50,,"fair enough, but i think the consequences of f...",1
2,train_subject4879,2014-02-11 04:36:02,,"yeah, i got 30/60. our marks were also 6 point...",1
3,train_subject4879,2014-02-08 04:28:31,,"try redreader beta, not super popular, but awe...",1
4,train_subject4879,2014-02-05 01:21:55,,i think you still get the breadth requirement....,1
...,...,...,...,...,...
21693,train_subject7367,2013-01-14 23:29:37,,What restaurant? I live in phoenix.,1
21694,train_subject7367,2012-12-28 02:27:53,,What is my ship doing on trees oh,1
21695,train_subject7367,2012-12-27 16:39:40,,/hug \nThis sounds like a dangerous position. ...,1
21696,train_subject7367,2012-12-27 15:34:30,,The thing about being diagnosed is that it sol...,1


In [8]:
df_neg

,User,Date,Title,Text,Label
5,train_subject8008,2013-09-17 16:20:51,,Such precision.,0
7,train_subject8008,2013-09-17 16:02:32,,She looks ravishing.,0
16,train_subject8008,2013-09-11 15:19:06,,http://imgur.com/5zaHE9v,0
32,train_subject5448,2015-06-21 23:52:24,,"As everyone is saying, it sucks, but with time...",0
33,train_subject5448,2015-06-16 18:18:32,,The Doctor can get through anything. He always...,0
...,...,...,...,...,...
264165,train_subject2605,2013-08-01 04:20:15,,Sweet! I'll start a thread on r/Atlanta this w...,0
264166,train_subject2605,2013-07-31 21:26:42,,I'm thinking about organizing a Reddit meet-up...,0
264167,train_subject2605,2013-07-31 21:20:51,,I was considering organizing a Reddit meet-up ...,0
264168,train_subject2605,2013-07-31 16:48:57,,All I used was a pair of scissors and my hands...,0


In [17]:
#bi-gram for text in df
from nltk import ngrams
from collections import Counter

def get_ngrams(text, n):
    n_grams = ngrams(nltk.word_tokenize(text), n)
    return [' '.join(grams) for grams in n_grams]

pos_text = [ get_ngrams(text, 2) for text in df['Text']]
neg_text = [ get_ngrams(text, 2) for text in df_neg['Text']]

In [19]:
from nltk import word_tokenize
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.corpus import stopwords

#flatten the list pos_text and neg_text
pos_text = [item for sublist in pos_text for item in sublist]
neg_text = [item for sublist in neg_text for item in sublist]

# Remove top 1000 most common words in negative_words in positive_words
freq_neg = pd.Series(' '.join(df_neg['Text']).split()).value_counts()[:2000]
freq_neg = list(freq_neg.index)
positive_words = [w for w in pos_text if w not in freq_neg]

#calculate multual information for positive_words
finder = BigramCollocationFinder.from_words(positive_words)
measures = BigramAssocMeasures()
finder.nbest(measures.pmi, 100)


[('! -spins', '-spins WWE'),
 ('! ..', '.. a'),
 ('! ...................', '................... okay'),
 ('! /u/Cyrus224', 'lol Steam'),
 ('! 111', '111 !'),
 ('! 15/f', '15/f here'),
 ('! 150,000', '150,000 miles'),
 ('! 1e1', '1e1 !'),
 ('! 1s3USsajJK3InKeO5ET4N8iA', '1s3USsajJK3InKeO5ET4N8iA !'),
 ('! 2e0', '2e0 )'),
 ('! 3m2', '3m2 !'),
 ('! 3m4', '3m4 !'),
 ('! 608', '608 North'),
 ('! AJDKSJFS', 'Aha best'),
 ('! ALL', 'ALL MAKES'),
 ('! AM', 'AM !'),
 ('! AWESOME', 'AWESOME I'),
 ('! Add', "Add 's"),
 ('! Adding', 'Adding you'),
 ('! AmLactin', 'AmLactin is'),
 ('! Americans', 'Americans accept'),
 ('! Annoyingly', 'Annoyingly they'),
 ('! Asian', 'Asian women'),
 ('! Ask', 'Ask anyone'),
 ('! Asterisk', 'Asterisk !'),
 ('! Astro', 'Astro !'),
 ('! Awwexplosion', 'Awwexplosion ;'),
 ('! B-team', 'B-team !'),
 ('! BATMAN', 'BATMAN PREPTIME'),
 ('! BRING', 'BRING BACK'),
 ('! Balloons', 'Balloons scare'),
 ('! Beat', 'Beat women'),
 ('! Beautiful', 'Beautiful T'),
 ('! Beautifully